In [4]:
# 1. 데이터 및 환경설정 (T5 기반 Dialogue Summarization)

import os
import re
import json
import yaml
from glob import glob
from pprint import pprint

import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl  # 쓰지 않더라도 requirements 맞추기용
from rouge import Rouge

from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback,
)

import wandb  # 선택: 학습 로깅용

# =========================================
# 1-1. 기본 설정: T5 한국어 요약 모델 선택
# =========================================
MODEL_NAME = "eenzeenee/t5-base-korean-summarization"  # T5-base 한국어 요약 모델

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# T5 계열은 보통 bos_token이 없으므로 None 방지
bos_token = tokenizer.bos_token if tokenizer.bos_token is not None else ""  # decoder input 앞에 붙일 토큰 (없어도 됨)
eos_token = tokenizer.eos_token if tokenizer.eos_token is not None else ""  # 보통 "</s>"

# =========================================
# 1-2. YAML Config 구성
# =========================================
config_data = {
    "general": {
        "data_path": "./data/",           # train.csv, dev.csv, test.csv 위치
        "model_name": MODEL_NAME,          # 사용할 T5 모델
        "output_dir": "./t5_outputs",      # 체크포인트 저장 디렉토리
    },
    "tokenizer": {
        "encoder_max_len": 512,            # 대화가 꽤 길 수 있어서 512 유지
        "decoder_max_len": 80,             # 요약은 상대적으로 짧게
        "bos_token": bos_token,
        "eos_token": eos_token,
        # 대화 내 특수 토큰을 보존하기 위해 special_tokens 등록
        "special_tokens": [
            "#Person1#", "#Person2#", "#Person3#",
            "#Person4#", "#Person5#", "#Person6#", "#Person7#",
            "#PhoneNumber#", "#Address#", "#PassportNumber#"
        ],
        # T5 요약 프롬프트
        "t5_prefix": "summarize: "
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 10,            # KoBART 20ep → T5 8~12ep 권장, 일단 10
        "learning_rate": 5e-5,             # 요약 태스크용으로 자주 쓰는 lr
        "per_device_train_batch_size": 8,  # OOM 방지 & 안정적인 학습
        "per_device_eval_batch_size": 16,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": "cosine",
        "optim": "adamw_torch",
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": "epoch",
        "save_strategy": "epoch",
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,     # ROUGE 계산을 위해 generate 사용
        "generation_max_length": 80,
        "metric_for_best_model": "rouge-l",  # ROUGE-L 기준으로 best ckpt 선택
        "greater_is_better": True,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "none",             # "wandb" 또는 "none"
    },
    "wandb": {
        "entity": "quriquri7",    # 예: "quriquri7"
        "project": "dialogue-summ-t5",     # 원하는 프로젝트명
        "name": "t5-base-run1"            # run 이름
    },
    "inference": {
        "ckt_path": "추론에_사용할_체크포인트_경로_또는_output_dir",  # 예: "t5_outputs/checkpoint-xxxx"
        "result_path": "./prediction_t5/",
        "no_repeat_ngram_size": 3,        # 반복 감소 → ROUGE 상승에 도움
        "early_stopping": True,
        "generate_max_length": 80,
        "num_beams": 5,                   # 빔 서치; 4~8 범위 튜닝 추천
        "batch_size": 32,
        "remove_tokens": [
            "<usr>",
            bos_token,
            eos_token,
            tokenizer.pad_token if tokenizer.pad_token is not None else "",
        ],
    },
}

# =========================================
# 1-3. Config 저장 및 로드
# =========================================
os.makedirs("./", exist_ok=True)
config_path = "./config_t5.yaml"

with open(config_path, "w", encoding="utf-8") as f:
    yaml.dump(config_data, f, allow_unicode=True)

with open(config_path, "r", encoding="utf-8") as f:
    loaded_config = yaml.safe_load(f)

print("=== Loaded Config ===")
pprint(loaded_config["general"])
pprint(loaded_config["tokenizer"])
pprint(loaded_config["training"])
pprint(loaded_config["wandb"])
pprint(loaded_config["inference"])

# =========================================
# 1-4. 데이터 로드 확인
# =========================================
data_path = loaded_config["general"]["data_path"]

train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
val_df   = pd.read_csv(os.path.join(data_path, "dev.csv"))

print("\n=== Train Sample ===")
print(train_df.tail())
print("\n=== Dev Sample ===")
print(val_df.tail())


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


=== Loaded Config ===
{'data_path': './data/',
 'model_name': 'eenzeenee/t5-base-korean-summarization',
 'output_dir': './t5_outputs'}
{'bos_token': '',
 'decoder_max_len': 80,
 'encoder_max_len': 512,
 'eos_token': '</s>',
 'special_tokens': ['#Person1#',
                    '#Person2#',
                    '#Person3#',
                    '#Person4#',
                    '#Person5#',
                    '#Person6#',
                    '#Person7#',
                    '#PhoneNumber#',
                    '#Address#',
                    '#PassportNumber#'],
 't5_prefix': 'summarize: '}
{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'generation_max_length': 80,
 'gradient_accumulation_steps': 1,
 'greater_is_better': True,
 'learning_rate': 5e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'metric_for_b

In [5]:
# 2. 데이터 가공 및 데이터셋 클래스 구축 (T5 summarization용)

class Preprocess:
    """
    - train/dev:
        encoder_input  : "summarize: " + dialogue
        decoder_input  : (teacher forcing용) bos_token + summary  (bos_token이 ""이면 summary 그대로)
        decoder_output : summary + eos_token
    - test:
        encoder_input  : "summarize: " + dialogue
        decoder_input  : bos_token (dummy, T5는 labels만 있어도 동작하지만 구조 통일)
    """
    def __init__(self, bos_token: str, eos_token: str, t5_prefix: str = "summarize: "):
        self.bos_token = bos_token
        self.eos_token = eos_token
        self.prefix = t5_prefix

    @staticmethod
    def make_set_as_df(file_path: str, is_train: bool = True):
        df = pd.read_csv(file_path)
        if is_train:
            return df[["fname", "dialogue", "summary"]]
        else:
            return df[["fname", "dialogue"]]

    def make_input(self, dataset: pd.DataFrame, is_test: bool = False):
        # encoder 입력에는 항상 T5 프리픽스를 붙여줌
        encoder_input = dataset["dialogue"].apply(
            lambda x: self.prefix + str(x)
        )

        if is_test:
            decoder_input = [self.bos_token] * len(dataset["dialogue"])
            return encoder_input.tolist(), decoder_input
        else:
            # Teacher forcing 용
            decoder_input = dataset["summary"].apply(
                lambda x: self.bos_token + str(x)
            )
            decoder_output = dataset["summary"].apply(
                lambda x: str(x) + self.eos_token
            )
            return (
                encoder_input.tolist(),
                decoder_input.tolist(),
                decoder_output.tolist(),
            )

# ============================
# Dataset 클래스들
# ============================

class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, length):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels    # dict or tensor
        self.length = length

    def __getitem__(self, idx):
        # encoder
        item = {
            key: val[idx].clone().detach()
            for key, val in self.encoder_input.items()
        }  # input_ids, attention_mask

        # decoder (teacher forcing)
        dec = {
            key: val[idx].clone().detach()
            for key, val in self.decoder_input.items()
        }
        dec["decoder_input_ids"] = dec["input_ids"]
        dec["decoder_attention_mask"] = dec["attention_mask"]
        dec.pop("input_ids")
        dec.pop("attention_mask")

        item.update(dec)
        # 🔥 여기에서 -100이 이미 들어간 labels 사용
        item["labels"] = self.labels["input_ids"][idx]  # T5는 labels만 있으면 내부에서 shift
        return item

    def __len__(self):
        return self.length


class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, length):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach()
            for key, val in self.encoder_input.items()
        }
        dec = {
            key: val[idx].clone().detach()
            for key, val in self.decoder_input.items()
        }
        dec["decoder_input_ids"] = dec["input_ids"]
        dec["decoder_attention_mask"] = dec["attention_mask"]
        dec.pop("input_ids")
        dec.pop("attention_mask")

        item.update(dec)
        # 🔥 여기에서 -100이 이미 들어간 labels 사용
        item["labels"] = self.labels["input_ids"][idx]
        return item

    def __len__(self):
        return self.length


class DatasetForInference(Dataset):
    def __init__(self, encoder_input, test_id, length):
        self.encoder_input = encoder_input
        self.test_id = test_id
        self.length = length

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach()
            for key, val in self.encoder_input.items()
        }
        item["ID"] = self.test_id[idx]
        return item

    def __len__(self):
        return self.length


# ============================
# Train/Val Dataset 생성 함수
# ============================

def prepare_train_dataset(config, preprocessor: Preprocess, data_path: str, tokenizer):
    train_file_path = os.path.join(data_path, "train.csv")
    val_file_path   = os.path.join(data_path, "dev.csv")

    train_data = preprocessor.make_set_as_df(train_file_path, is_train=True)
    val_data   = preprocessor.make_set_as_df(val_file_path,   is_train=True)

    print("-" * 150)
    print(f"[Train dialogue]\n{train_data['dialogue'][0]}")
    print(f"[Train summary]\n{train_data['summary'][0]}")
    print("-" * 150)
    print(f"[Dev dialogue]\n{val_data['dialogue'][0]}")
    print(f"[Dev summary]\n{val_data['summary'][0]}")

    encoder_input_train, decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val,   decoder_input_val,   decoder_output_val   = preprocessor.make_input(val_data)

    print("-" * 10, "Load data complete", "-" * 10)

    # ----- Train tokenization -----
    tokenized_encoder_inputs = tokenizer(
        encoder_input_train,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"],
        return_token_type_ids=False,
    )
    tokenized_decoder_inputs = tokenizer(
        decoder_input_train,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )
    tokenized_decoder_outputs = tokenizer(
        decoder_output_train,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )
    # 🔥 [중요] labels에서 pad_token_id를 -100으로 바꿔주기
    labels_train = tokenized_decoder_outputs["input_ids"].clone()
    labels_train[labels_train == tokenizer.pad_token_id] = -100

    train_dataset = DatasetForTrain(
        tokenized_encoder_inputs,
        tokenized_decoder_inputs,
        {"input_ids": labels_train},   # ← labels dict로 감싸줌
        len(encoder_input_train),
    )

    # ----- Val tokenization -----
    val_tokenized_encoder_inputs = tokenizer(
        encoder_input_val,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"],
        return_token_type_ids=False,
    )
    val_tokenized_decoder_inputs = tokenizer(
        decoder_input_val,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )
    val_tokenized_decoder_outputs = tokenizer(
        decoder_output_val,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )
    labels_val = val_tokenized_decoder_outputs["input_ids"].clone()
    labels_val[labels_val == tokenizer.pad_token_id] = -100

    val_dataset = DatasetForVal(
        val_tokenized_encoder_inputs,
        val_tokenized_decoder_inputs,
        {"input_ids": labels_val},   # ← labels dict로 감싸줌
        len(encoder_input_val),
    )

    print("-" * 10, "Make dataset complete", "-" * 10)
    return train_dataset, val_dataset


In [8]:
# 3. Trainer 및 TrainingArguments 구축하기 (T5 버전)

def compute_metrics(config, tokenizer, pred):
    """
    - 대회 평가는 형태소 단위 토크나이즈 후 ROUGE를 계산하지만,
      여기서는 문장 단위 token 기반 ROUGE를 사용.
    - 대신 remove_tokens로 <usr>, BOS/EOS, PAD 등은 정리해 줌.
    """
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    # -100을 pad_token_id로 복구
    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # 불필요 토큰 제거
    remove_tokens = config["inference"]["remove_tokens"]
    replaced_preds = decoded_preds.copy()
    replaced_labels = decoded_labels.copy()

    for token in remove_tokens:
        if token:
            replaced_preds  = [sent.replace(token, " ") for sent in replaced_preds]
            replaced_labels = [sent.replace(token, " ") for sent in replaced_labels]

    # 디버깅용 샘플 출력
    print("-" * 150)
    print(f"[PRED 0]: {replaced_preds[0]}")
    print(f"[GOLD 0]: {replaced_labels[0]}")
    if len(replaced_preds) > 1:
        print("-" * 150)
        print(f"[PRED 1]: {replaced_preds[1]}")
        print(f"[GOLD 1]: {replaced_labels[1]}")
    if len(replaced_preds) > 2:
        print("-" * 150)
        print(f"[PRED 2]: {replaced_preds[2]}")
        print(f"[GOLD 2]: {replaced_labels[2]}")

    # ROUGE 계산
    results = rouge.get_scores(replaced_preds, replaced_labels, avg=True)
    result = {key: value["f"] for key, value in results.items()}  # f1 score

    # Trainer에서 metric_for_best_model="rouge-l"로 사용
    return result


def load_trainer_for_train(config, model, tokenizer, train_dataset, val_dataset):
    print("-" * 10, "Make training arguments", "-" * 10)

    training_args = Seq2SeqTrainingArguments(
        output_dir=config["general"]["output_dir"],
        overwrite_output_dir=config["training"]["overwrite_output_dir"],
        num_train_epochs=config["training"]["num_train_epochs"],
        learning_rate=config["training"]["learning_rate"],
        per_device_train_batch_size=config["training"]["per_device_train_batch_size"],
        per_device_eval_batch_size=config["training"]["per_device_eval_batch_size"],
        warmup_ratio=config["training"]["warmup_ratio"],
        weight_decay=config["training"]["weight_decay"],
        lr_scheduler_type=config["training"]["lr_scheduler_type"],
        optim=config["training"]["optim"],
        gradient_accumulation_steps=config["training"]["gradient_accumulation_steps"],
        evaluation_strategy=config["training"]["evaluation_strategy"],
        save_strategy=config["training"]["save_strategy"],
        save_total_limit=config["training"]["save_total_limit"],
        fp16=config["training"]["fp16"],
        load_best_model_at_end=config["training"]["load_best_model_at_end"],
        seed=config["training"]["seed"],
        logging_dir=config["training"]["logging_dir"],
        logging_strategy=config["training"]["logging_strategy"],
        predict_with_generate=config["training"]["predict_with_generate"],
        generation_max_length=config["training"]["generation_max_length"],
        do_train=config["training"]["do_train"],
        do_eval=config["training"]["do_eval"],
        report_to=config["training"]["report_to"],  # "wandb" or "none"
        metric_for_best_model=config["training"]["metric_for_best_model"],
        greater_is_better=config["training"]["greater_is_better"],
        generation_num_beams=config["inference"]["num_beams"],
        # 옵션: label_smoothing_factor=0.1  # transformers 버전에 따라 사용 가능
        # 🔥 wandb 완전 비활성화
        
    )

    # ----- wandb 설정 -----
    #USE_WANDB = config["training"]["report_to"] == "wandb"
    #if USE_WANDB:
    #    try:
    #        wandb.init(
    #            entity=config["wandb"]["entity"],
    #            project=config["wandb"]["project"],
    #            name=config["wandb"]["name"],
    #        )
    #        os.environ["WANDB_LOG_MODEL"] = "true"
    #        os.environ["WANDB_WATCH"] = "false"
    #    except Exception as e:
    #        print("⚠️ wandb init 실패, wandb 없이 진행합니다:", e)
    #        os.environ["WANDB_DISABLED"] = "true"
    #        config["training"]["report_to"] = "none"    
    #else:
    #    os.environ["WANDB_DISABLED"] = "true"
    #    print("🚫 wandb 비활성화 상태로 학습을 진행합니다.")

    # ----- EarlyStopping -----
    early_stop_cb = EarlyStoppingCallback(
        early_stopping_patience=config["training"]["early_stopping_patience"],
        early_stopping_threshold=config["training"]["early_stopping_threshold"],
    )

    print("-" * 10, "Make trainer", "-" * 10)

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=lambda pred: compute_metrics(config, tokenizer, pred),
        callbacks=[early_stop_cb],
    )

    print("-" * 10, "Make trainer complete", "-" * 10)
    return trainer


def load_tokenizer_and_model_for_train(config, device):
    print("-" * 10, "Load tokenizer & T5 model", "-" * 10)
    model_name = config["general"]["model_name"]

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 대화 내 특수 토큰 추가
    special_tokens_dict = {"additional_special_tokens": config["tokenizer"]["special_tokens"]}
    tokenizer.add_special_tokens(special_tokens_dict)

    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # 토크나이저 크기 변경 반영
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)

    print(model.config)
    print("-" * 10, "Load tokenizer & model complete", "-" * 10)
    return model, tokenizer


In [9]:
# 4. 모델 학습하기 (Train Loop)

def main_train(config):
    # 4-1. device 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("-" * 10, f"device : {device}", "-" * 10)
    print(torch.__version__)

    # 4-2. T5 모델 & 토크나이저 로드
    model, tokenizer = load_tokenizer_and_model_for_train(config, device)
    print("-" * 10, "tokenizer special tokens : ", tokenizer.special_tokens_map, "-" * 10)

    # 4-3. 데이터셋 생성
    preprocessor = Preprocess(
        bos_token=config["tokenizer"]["bos_token"],
        eos_token=config["tokenizer"]["eos_token"],
        t5_prefix=config["tokenizer"]["t5_prefix"],
    )
    data_path = config["general"]["data_path"]

    train_dataset, val_dataset = prepare_train_dataset(
        config, preprocessor, data_path, tokenizer
    )

    # 4-4. Trainer 생성 및 학습
    trainer = load_trainer_for_train(
        config,
        model,
        tokenizer,
        train_dataset,
        val_dataset,
    )

    trainer.train()

    # 4-5. wandb 종료 (선택)
    #if config["training"]["report_to"] == "wandb":
    #    wandb.finish()

    # best 모델은 TrainingArguments.load_best_model_at_end=True에 의해 로딩됨
    print("✅ 학습 완료")

     # 🔍 여기서부터 "dev 샘플 1개 직접 요약해보기" 디버깅 코드
    print("\n=== Dev 샘플 요약 테스트 ===")
    sample = val_df.iloc[0]  # 1번 셀에서 로드한 val_df 사용

    text = config["tokenizer"]["t5_prefix"] + sample["dialogue"]
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"]
    ).to(device)

    output_ids = model.generate(
        **inputs,
        max_length=80,
        num_beams=5,
        no_repeat_ngram_size=3,
    )

    gen = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print("GEN :", gen)
    print("GOLD:", sample["summary"])


if __name__ == "__main__":
    # 위 1번 셀에서 만든 loaded_config를 그대로 사용한다고 가정
    main_train(loaded_config)


---------- device : cuda ----------
2.1.2+cu118
---------- Load tokenizer & T5 model ----------


/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


T5Config {
  "_name_or_path": "eenzeenee/t5-base-korean-summarization",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 128,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 50368
}

---------- Load tokenizer & model complete ----------
---------- tokenizer special tokens :  {'eos_token': '</s>', 'unk_token': '<pad>', 'pad_token': '<pad>',

/data/ephemeral/home/nlp_server/venv310/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


---------- Make dataset complete ----------
---------- Make training arguments ----------
---------- Make trainer ----------
---------- Make trainer complete ----------


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,1.987500,0.004187,0.068461,0.000959,0.068461
2,0.008300,0.000631,0.002456,0.000000,0.002456
3,0.003000,0.000340,0.007544,0.000000,0.007544


------------------------------------------------------------------------------------------------------------------------------------------------------
[PRED 0]:  감감기에                                                                                                                            
[GOLD 0]: #Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하고, 천식 검사를 위해 폐 전문의에게 가볼 것을 권합니다.                                                                                     
------------------------------------------------------------------------------------------------------------------------------------------------------
[PRED 1]:  JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ                                                
[GOLD 1]: #Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.                                                                                                     
-------------------------------------------------------------------------------------

KeyboardInterrupt: 

In [ ]:
# 5. 모델 추론하기 (Inference)

# 추론에 사용할 checkpoint 설정 (필수!)
loaded_config["inference"]["ckt_path"] = "t5_outputs/checkpoint-xxxx"  # 실제 ckpt 경로로 교체 필요


def prepare_test_dataset(config, preprocessor: Preprocess, tokenizer):
    test_file_path = os.path.join(config["general"]["data_path"], "test.csv")

    test_data = preprocessor.make_set_as_df(test_file_path, is_train=False)
    test_id = test_data["fname"]

    print("-" * 150)
    print(f"[Test dialogue sample]\n{test_data['dialogue'][0]}")
    print("-" * 150)

    encoder_input_test, decoder_input_test = preprocessor.make_input(
        test_data, is_test=True
    )

    print("-" * 10, "Load test data complete", "-" * 10)

    test_tokenized_encoder_inputs = tokenizer(
        encoder_input_test,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["encoder_max_len"],
        return_token_type_ids=False,
    )
    test_tokenized_decoder_inputs = tokenizer(
        decoder_input_test,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=config["tokenizer"]["decoder_max_len"],
        return_token_type_ids=False,
    )

    test_dataset = DatasetForInference(
        test_tokenized_encoder_inputs,
        test_id,
        len(encoder_input_test),
    )

    print("-" * 10, "Make test dataset complete", "-" * 10)
    return test_data, test_dataset


def load_tokenizer_and_model_for_test(config, device):
    print("-" * 10, "Load tokenizer & model for inference", "-" * 10)

    model_name = config["general"]["model_name"]
    ckt_path = config["inference"]["ckt_path"]

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {"additional_special_tokens": config["tokenizer"]["special_tokens"]}
    tokenizer.add_special_tokens(special_tokens_dict)

    model = T5ForConditionalGeneration.from_pretrained(ckt_path)
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    model.eval()

    print("-" * 10, "Load tokenizer & model complete", "-" * 10)
    return model, tokenizer


def inference(config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("-" * 10, f"device : {device}", "-" * 10)
    print(torch.__version__)

    model, tokenizer = load_tokenizer_and_model_for_test(config, device)

    preprocessor = Preprocess(
        bos_token=config["tokenizer"]["bos_token"],
        eos_token=config["tokenizer"]["eos_token"],
        t5_prefix=config["tokenizer"]["t5_prefix"],
    )

    test_data, test_dataset = prepare_test_dataset(config, preprocessor, tokenizer)
    dataloader = DataLoader(
        test_dataset,
        batch_size=config["inference"]["batch_size"],
    )

    summaries = []
    text_ids = []

    no_repeat_ngram_size = config["inference"]["no_repeat_ngram_size"]
    early_stopping = config["inference"]["early_stopping"]
    max_length = config["inference"]["generate_max_length"]
    num_beams = config["inference"]["num_beams"]

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            text_ids.extend(batch["ID"])

            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                no_repeat_ngram_size=no_repeat_ngram_size,
                early_stopping=early_stopping,
                max_length=max_length,
                num_beams=num_beams,
            )

            decoded = tokenizer.batch_decode(
                generated_ids,
                skip_special_tokens=False,
            )
            summaries.extend(decoded)

    # 불필요 스페셜 토큰 제거
    remove_tokens = config["inference"]["remove_tokens"]
    cleaned_summaries = summaries.copy()
    for token in remove_tokens:
        if token:
            cleaned_summaries = [
                sent.replace(token, " ") for sent in cleaned_summaries
            ]

    output = pd.DataFrame(
        {
            "fname": test_data["fname"],
            "summary": cleaned_summaries,
        }
    )

    result_path = config["inference"]["result_path"]
    os.makedirs(result_path, exist_ok=True)

    save_path = os.path.join(result_path, "output_t5.csv")
    output.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"✅ Inference 결과 저장 완료: {save_path}")

    return output


if __name__ == "__main__":
    output_df = inference(loaded_config)
    output_df.head()
